<a href="https://colab.research.google.com/github/AnchitSharma/cuistomer-segmentation/blob/master/customer_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Data manipulation Libraries
import numpy as np
import pandas as pd
# Data Viz
import matplotlib.pyplot as plt
# Scikit-Learn ML Libraries
from sklearn.preprocessing import *
from sklearn.model_selection import *
from sklearn.metrics import *

In [2]:
# Boosting Algorithm Libraries
from xgboost import XGBClassifier
# from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold, KFold

Load Data

https://www.kaggle.com/vetrirah/customer

In [3]:
train = pd.read_csv("https://raw.githubusercontent.com/AnchitSharma/cuistomer-segmentation/master/customers_data/Train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/AnchitSharma/cuistomer-segmentation/master/customers_data/Test.csv")
sub = pd.read_csv("https://raw.githubusercontent.com/AnchitSharma/cuistomer-segmentation/master/customers_data/sample_submission.csv")

In [4]:
train.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


Perform EDA (Exploratory Data Analysis) 

Explore Test and Train Data and get to know what each Column/Feature denotes

In [5]:
def display_data_information(data, data_types, dataframe_name):
  print("Information of ", dataframe_name, "Rows = ", data.shape[0], "| columns = ", data.shape[1], "\n")
  data.info()

  print("\n")
  for VARIABLE in data_types:
    data_type = data.select_dtypes(include=[ VARIABLE ]).dtypes

    if len(data_type) > 0:
      print(str(len(data_type)) + " "+VARIABLE+" Features\n"+str(data_type)+"\n" )


# Display Data Information of "train: :
data_types = ["float32", "float64", "int32", "int64", "object", "category", "datetime64[ns]"]
display_data_information(train, data_types, "train")

Information of  train Rows =  8068 | columns =  11 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   object 
 2   Ever_Married     7928 non-null   object 
 3   Age              8068 non-null   int64  
 4   Graduated        7990 non-null   object 
 5   Profession       7944 non-null   object 
 6   Work_Experience  7239 non-null   float64
 7   Spending_Score   8068 non-null   object 
 8   Family_Size      7733 non-null   float64
 9   Var_1            7992 non-null   object 
 10  Segmentation     8068 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 693.5+ KB


2 float64 Features
Work_Experience    float64
Family_Size        float64
dtype: object

2 int64 Features
ID     int64
Age    int64
dtype: object

7 object Features
Gender      

In [6]:
# train.select_dtypes(include=["float64"]).dtypes
display_data_information(test, data_types, "test")

Information of  test Rows =  2627 | columns =  10 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2627 entries, 0 to 2626
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               2627 non-null   int64  
 1   Gender           2627 non-null   object 
 2   Ever_Married     2577 non-null   object 
 3   Age              2627 non-null   int64  
 4   Graduated        2603 non-null   object 
 5   Profession       2589 non-null   object 
 6   Work_Experience  2358 non-null   float64
 7   Spending_Score   2627 non-null   object 
 8   Family_Size      2514 non-null   float64
 9   Var_1            2595 non-null   object 
dtypes: float64(2), int64(2), object(6)
memory usage: 205.4+ KB


2 float64 Features
Work_Experience    float64
Family_Size        float64
dtype: object

2 int64 Features
ID     int64
Age    int64
dtype: object

6 object Features
Gender            object
Ever_Married      object
Graduated

Display head and tail of the dataframe

In [7]:
def display_head_tail(data, head_rows, tail_rows):
  display("Data Head & Tail : ")
  display(data.head(head_rows).append(data.tail(tail_rows)))

display_head_tail(train, head_rows=3, tail_rows=2)

'Data Head & Tail : '

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
8066,467299,Female,No,27,Yes,Healthcare,1.0,Low,4.0,Cat_6,B
8067,461879,Male,Yes,37,Yes,Executive,0.0,Average,3.0,Cat_4,B


In [8]:
display_head_tail(test, head_rows=3, tail_rows=2)

'Data Head & Tail : '

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1
0,458989,Female,Yes,36,Yes,Engineer,0.0,Low,1.0,Cat_6
1,458994,Male,Yes,37,Yes,Healthcare,8.0,Average,4.0,Cat_6
2,458996,Female,Yes,69,No,NaN,0.0,Low,1.0,Cat_6
2625,467961,Male,Yes,47,Yes,Executive,1.0,High,5.0,Cat_4
2626,467968,Female,No,43,Yes,Healthcare,9.0,Low,3.0,Cat_7


In [9]:
def display_data_description(data, numeric_data_types, categorical_data_type):
  print(" Data Description ")
  display(data.describe(include =  numeric_data_types))
  print(" ")
  display(data.describe(include = categorical_data_type))

In [10]:
display_data_description(train, data_types[0:4], data_types[4:7])

 Data Description 


,ID,Age,Work_Experience,Family_Size
count,8068.000000,8068.000000,7239.000000,7733.000000
mean,463479.214551,43.466906,2.641663,2.850123
std,2595.381232,16.711696,3.406763,1.531413
min,458982.000000,18.000000,0.000000,1.000000
25%,461240.750000,30.000000,0.000000,2.000000
50%,463472.500000,40.000000,1.000000,3.000000
75%,465744.250000,53.000000,4.000000,4.000000
max,467974.000000,89.000000,14.000000,9.000000


,Gender,Ever_Married,Graduated,Profession,Spending_Score,Var_1,Segmentation
count,8068,7928,7990,7944,8068,7992,8068
unique,2,2,2,9,3,7,4
top,Male,Yes,Yes,Artist,Low,Cat_6,D
freq,4417,4643,4968,2516,4878,5238,2268


In [11]:
display_data_description(test, data_types[0:4], data_types[4:7])

 Data Description 


,ID,Age,Work_Experience,Family_Size
count,2627.000000,2627.000000,2358.000000,2514.000000
mean,463433.918919,43.649791,2.552587,2.825378
std,2618.245698,16.967015,3.341094,1.551906
min,458989.000000,18.000000,0.000000,1.000000
25%,461162.500000,30.000000,0.000000,2.000000
50%,463379.000000,41.000000,1.000000,2.000000
75%,465696.000000,53.000000,4.000000,4.000000
max,467968.000000,89.000000,14.000000,9.000000


,Gender,Ever_Married,Graduated,Profession,Spending_Score,Var_1
count,2627,2577,2603,2589,2627,2595
unique,2,2,2,9,3,7
top,Male,Yes,Yes,Artist,Low,Cat_6
freq,1424,1520,1602,802,1616,1672


In [12]:
# Checking Percentage of common ID's between train and test data using Unique train values

print(np.intersect1d(train['ID'], test['ID']).shape[0]/train['ID'].nunique())

common_ids = len(
    set(test['ID'].unique()).intersection(set(train['ID'].unique()))
    )

print("Common IDs : ", common_ids)

# Data Leak as out of 2627 Rows , there are 2332 ID's in common

print("Unique IDs : ", test.shape[0] - common_ids)

0.2890431333663857
Common IDs :  2332
Unique IDs :  295


In [13]:
set(test['ID'].unique()).intersection?

Object `intersection` not found.


In [14]:
testx = pd.merge(test, train, how="left", on="ID")

Remove Duplicate Rows from Train data if present

In [15]:
def remove_duplicates(data):
  print("BEFORE REMOVING DUPLICATES - No of Rows : ", data.shape[0])
  data.drop_duplicates(keep="first", inplace=True)
  print("AFTER REMOVING DUPLICATES  - No of Rows : ", data.shape[0])

  return data

In [16]:
train = remove_duplicates(train)

BEFORE REMOVING DUPLICATES - No of Rows :  8068
AFTER REMOVING DUPLICATES  - No of Rows :  8068


Fill/Impute Missing values continuous - Mean/Median/Any specific value

& categorical - Others/ForwardFill/BackFill


In [17]:
# Let LightGBM classifier Handles the issues

In [18]:
def display_unique(data):

  for column in data.columns:
    print("No of Unique values in "+column+" Column are : "+str(data[column].nunique()) )# nunique gives count of unique values
    print("Actual Unique Values in "+column+" Column are : "+str(data[column].sort_values(ascending=True, na_position="last").unique()))

    print("NULL Values : ")
    print(data[ column ].isnull().sum())
    print("Value counts : ")
    print(data[column].value_counts())
    print(" ")

In [19]:
display_unique(train)

No of Unique values in ID Column are : 8068
Actual Unique Values in ID Column are : [458982 458983 458984 ... 467972 467973 467974]
NULL Values : 
0
Value counts : 
460799    1
466217    1
460042    1
462091    1
464144    1
         ..
463511    1
467609    1
461468    1
459421    1
460801    1
Name: ID, Length: 8068, dtype: int64
 
No of Unique values in Gender Column are : 2
Actual Unique Values in Gender Column are : ['Female' 'Male']
NULL Values : 
0
Value counts : 
Male      4417
Female    3651
Name: Gender, dtype: int64
 
No of Unique values in Ever_Married Column are : 2
Actual Unique Values in Ever_Married Column are : ['No' 'Yes' nan]
NULL Values : 
140
Value counts : 
Yes    4643
No     3285
Name: Ever_Married, dtype: int64
 
No of Unique values in Age Column are : 67
Actual Unique Values in Age Column are : [18 19 20 21 22 23 25 26 27 28 29 30 31 32 33 35 36 37 38 39 40 41 42 43
 45 46 47 48 49 50 51 52 53 55 56 57 58 59 60 61 62 63 65 66 67 68 69 70
 71 72 73 74 75 76 77 7

In [20]:
for i in train:
  print(f"column {i} unique values {train[i].unique()}")

column ID unique values [462809 462643 466315 ... 465406 467299 461879]
column Gender unique values ['Male' 'Female']
column Ever_Married unique values ['No' 'Yes' nan]
column Age unique values [22 38 67 40 56 32 33 61 55 26 19 70 58 41 31 79 49 18 36 35 45 42 83 27
 28 47 29 57 76 25 72 48 74 59 39 51 30 63 52 60 68 86 50 43 80 37 46 69
 78 71 82 23 20 85 21 53 62 75 65 89 66 73 77 87 84 81 88]
column Graduated unique values ['No' 'Yes' nan]
column Profession unique values ['Healthcare' 'Engineer' 'Lawyer' 'Entertainment' 'Artist' 'Executive'
 'Doctor' 'Homemaker' 'Marketing' nan]
column Work_Experience unique values [ 1. nan  0.  4.  9. 12.  3. 13.  5.  8. 14.  7.  2.  6. 10. 11.]
column Spending_Score unique values ['Low' 'Average' 'High']
column Family_Size unique values [ 4.  3.  1.  2.  6. nan  5.  8.  7.  9.]
column Var_1 unique values ['Cat_4' 'Cat_6' 'Cat_7' 'Cat_3' 'Cat_1' 'Cat_2' nan 'Cat_5']
column Segmentation unique values ['D' 'A' 'B' 'C']


Multi class classification problem - Target has more than 2 categories

Target - Segmentation has 4 values of customers ['D', 'A', 'B', 'C']

# Feature Engineering

Feature Selection - Selection of most important existing features

Feature Creation - Creation of New Features from the Existing Features/ Predictors :

In [21]:
# concatenate train and test data into single dataframe

train["is_train"] = 1
test["is_train"] = 0
df = pd.concat([train, test])

In [22]:
df.columns

Index(['ID', 'Gender', 'Ever_Married', 'Age', 'Graduated', 'Profession',
       'Work_Experience', 'Spending_Score', 'Family_Size', 'Var_1',
       'Segmentation', 'is_train'],
      dtype='object')

In [23]:
# Label Encode Technique

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for i in ["Gender", "Ever_Married", "Graduated", "Profession", "Spending_Score", "Var_1"]:
  df[i] = le.fit_transform(df[i].astype('str'))


In [24]:
df.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation,is_train
0,462809,1,0,22,0,5,1.0,2,4.0,3,D,1
1,462643,0,1,38,1,2,NaN,0,3.0,3,A,1
2,466315,0,1,67,1,2,1.0,2,1.0,5,B,1
3,461735,1,1,67,1,7,0.0,1,2.0,5,B,1
4,462669,0,1,40,1,3,NaN,1,6.0,5,A,1


In [25]:
df['Segmentation'] = df['Segmentation'].map({'A':0, 'B':1, 'C':2, 'D':3})

Handle missing values

* Family_Size , Work_Experience 


In [26]:
df['Family_Size'].isna().sum()#.value_counts()
df['Family_Size'].value_counts()
# fill the na values with 2.0 a minimum family is 2
df['Family_Size'] = df['Family_Size'].fillna(2.0)

In [27]:
df['Work_Experience'].isna().sum()
df['Work_Experience'].value_counts()
# filling na values with 0.0 default Work_Experience
df['Work_Experience'] = df['Work_Experience'].fillna(0.0)

In [28]:
df['Work_Experience'].isna().sum()

0

In [29]:
# Get back train data from df

train = df[df['is_train'] == 1]
test = train = df[df['is_train'] == 0]

Split Train Data into Train and Validation Data with

Predictors(Independent) & Target(Dependent)

In [30]:
predictor_train = train.drop(['Segmentation', 'is_train', 'ID'], axis=1)
target_train = train['Segmentation']

In [31]:
predictor_test = test.drop(['is_train', 'ID'], axis=1)

In [32]:
predictor_test.head()

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,0,1,36,1,2,0.0,2,1.0,5,NaN
1,1,1,37,1,5,8.0,0,4.0,5,NaN
2,0,1,69,0,9,0.0,2,1.0,5,NaN
3,1,1,59,0,4,11.0,1,2.0,5,NaN
4,0,0,19,0,8,0.0,2,4.0,5,NaN


## Create baseline ML model

In [80]:
predictor_train

Gender             0
Ever_Married       0
Age                0
Graduated          0
Profession         0
Work_Experience    0
Spending_Score     0
Family_Size        0
Var_1              0
dtype: int64

In [33]:
categorical_features = ["Gender", "Ever_Married" ,"Graduated" ,"Profession" ,"Spending_Score" ,"Var_1" ]

lgb_model = LGBMClassifier()

# Apply stratified K-Fold cross validation where K = 5 or n_splits=5

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)
acc = []

# Pass predictor_train, target_train for Cross Validation
for fold, (t_id, v_id) in enumerate(kf.split(predictor_train, target_train)):

  # Split train and validation data
  tx = predictor_train.iloc[t_id]; ty = target_train.iloc[t_id]
  vx = predictor_train.iloc[v_id]; vy = target_train.iloc[v_id]


  # Train/Fit the data to LightGBM model
  lgb_model.fit(tx, ty, categorical_features= categorical_features)

  # Predict the Validation  Data to Train LightGBM Model
  val_y = lgb_model.predict(vx)

  # Get Accuracy Score on validation data for each fold
  acc_score = accuracy_score(vy, val_y)
  acc.append(acc_score)
  print(f"fold {fold} accuracy {acc_score}")

print(f"Mean accuracy score {np.mean(acc)}")

ValueError: ignored

In [88]:
predictor_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2627 entries, 0 to 2626
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Gender           2627 non-null   int64  
 1   Ever_Married     2627 non-null   int64  
 2   Age              2627 non-null   int64  
 3   Graduated        2627 non-null   int64  
 4   Profession       2627 non-null   int64  
 5   Work_Experience  2627 non-null   float64
 6   Spending_Score   2627 non-null   int64  
 7   Family_Size      2627 non-null   float64
 8   Var_1            2627 non-null   int64  
dtypes: float64(2), int64(7)
memory usage: 205.2 KB


In [63]:
display_unique(predictor_train)

No of Unique values in Gender Column are : 2
Actual Unique Values in Gender Column are : [0 1]
NULL Values : 
0
Value counts : 
1    1424
0    1203
Name: Gender, dtype: int64
 
No of Unique values in Ever_Married Column are : 3
Actual Unique Values in Ever_Married Column are : [0 1 2]
NULL Values : 
0
Value counts : 
1    1520
0    1057
2      50
Name: Ever_Married, dtype: int64
 
No of Unique values in Age Column are : 67
Actual Unique Values in Age Column are : [18 19 20 21 22 23 25 26 27 28 29 30 31 32 33 35 36 37 38 39 40 41 42 43
 45 46 47 48 49 50 51 52 53 55 56 57 58 59 60 61 62 63 65 66 67 68 69 70
 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89]
NULL Values : 
0
Value counts : 
42    88
28    77
41    76
30    76
43    75
      ..
77     9
88     9
78     7
81     7
75     5
Name: Age, Length: 67, dtype: int64
 
No of Unique values in Graduated Column are : 3
Actual Unique Values in Graduated Column are : [0 1 2]
NULL Values : 
0
Value counts : 
1    1602
0    1001
2